# Parse SEC 10-Q

In [594]:
import pandas as pd
import numpy as np
import time, sys, os, re
import datetime, pickle, bz2
import getpass
from tqdm import tqdm
import datetime
from datetime import datetime

In [3]:
from __future__ import division, unicode_literals 
import codecs

# Import requests to retrive Web Urls example HTML. TXT 
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

# import re module for REGEXes
import re

In [490]:
with open("/home/dada/MA_Minus/10-Q/LCNB/LCNB-0") as fp:
    document= BeautifulSoup(fp.read()).get_text().lower()

In [561]:
# extract text between item 2 and item 3
regex = re.compile('(item 2)|(item 3)') 

# Use finditer to math the regex
matches = regex.finditer(document)

In [542]:
# Create the dataframe
test_df = pd.DataFrame([(x.start(), x.end()) for x in matches])

In [554]:
def cleanStr(test_df_1):
    test_df_1 = re.sub('\xa0', '', test_df_1)
    test_df_1 = re.sub('\n','', test_df_1)
    test_df_1 = re.sub('\n\n','', test_df_1)
    test_df_1 = re.sub('\n\n\n','', test_df_1)
    test_df_1 = re.sub('\t', '', test_df_1)
    test_df_1 = re.sub(' ·', '', test_df_1)
        
    return test_df_1

In [576]:
#parse 10-Q MA-
ma_neg = sorted(os.listdir("/home/dada/MA_Minus/10-Q"))
len(ma_neg)
ma_neg[:5]

['ACBI', 'ACNB', 'AMRB', 'ASRV', 'ATLO']

In [584]:
MA_Minus_df = pd.DataFrame()

for i in tqdm(ma_neg):
    
    i_list = sorted(os.listdir("/home/dada/MA_Minus/10-Q/{}".format(i)))
                    
    MA_Minus_i = pd.DataFrame()            
    for j in i_list:        
        with open("/home/dada/MA_Minus/10-Q/{}/{}".format(i, j)) as fp:
            document = BeautifulSoup(fp.read()).get_text().lower()

        if "item 2" in document:
            regex_i_j = regex.finditer(document)
            match_df = pd.DataFrame([(x.start(), x.end()) for x in regex_i_j])
            parse_i_j = BeautifulSoup(document[match_df.iloc[0,1]:match_df.iloc[len(match_df)-1,0]], 'lxml')
            parse_i_j = parse_i_j.get_text("\n\n")
            parse_i_j = cleanStr(parse_i_j)
            MA_Minus_i = MA_Minus_i.append(pd.DataFrame([{"Ticker":i,"Period":j,"MDAA":parse_i_j}]), ignore_index = True)
            
        else:
            continue
            
    MA_Minus_df = MA_Minus_df.append(MA_Minus_i)    

100%|██████████| 112/112 [35:59<00:00, 19.29s/it]


In [585]:
print(MA_Minus_df.shape)
MA_Minus_df.head()

(844, 3)


,Ticker,Period,MDAA
0,ACBI,ACBI-0,
0,ACBI,ACBI-1,.unregistered sales of equity securities and u...
0,ACBI,ACBI-2,.unregistered sales of equity securities and u...
0,ACBI,ACBI-3,.unregistered sales of equity securities and u...
0,ACBI,ACBI-4,.unregistered sales of equity securities and u...


In [586]:
#parse 10-Q MA+
ma_pos = sorted(os.listdir("/home/dada/MA_Plus/10-Q"))
len(ma_pos)

109

In [588]:
MA_Plus_df = pd.DataFrame()

for i in tqdm(ma_pos):
    
    i_list = sorted(os.listdir("/home/dada/MA_Plus/10-Q/{}".format(i)))
    
    MA_Plus_i = pd.DataFrame()  
    for j in i_list:
        
        with open("/home/dada/MA_Plus/10-Q/{}/{}".format(i, j)) as fp:
            document = BeautifulSoup(fp.read()).get_text().lower()

        if "item 2" in document:
            regex_i_j = regex.finditer(document)
            match_df = pd.DataFrame([(x.start(), x.end()) for x in regex_i_j])
            parse_i_j = BeautifulSoup(document[match_df.iloc[0,1]:match_df.iloc[len(match_df)-1,0]], 'lxml')
            parse_i_j = parse_i_j.get_text("\n\n")
            parse_i_j = cleanStr(parse_i_j)
            MA_Plus_i = MA_Plus_i.append(pd.DataFrame([{"Ticker":i,"Period":j,"MDAA":parse_i_j}]), ignore_index = True)
            
        else:
            continue
            
    MA_Plus_df = MA_Plus_df.append(MA_Plus_i)    

100%|██████████| 109/109 [36:02<00:00, 19.84s/it]


In [589]:
MA_Plus_df["MA"] = 1
MA_Minus_df["MA"] = 0

In [590]:
MA_all = MA_Minus_df.append(MA_Plus_df, ignore_index = True)

In [593]:
pickle.dump(MA_Plus_df, open("MA_Plus_df.pkl", 'wb'))
pickle.dump(MA_Minus_df, open("MA_Minus_df.pkl", 'wb'))

In [596]:
ls -al

total 228344
drwxr-xr-x  3 dada dada      4096 Aug 11 02:02  ./
drwxr-xr-x 14 dada dada      4096 Aug  4 10:10  ../
-rw-r--r--  1 dada dada     14323 Aug  2 22:42 'AI in finance proposal.docx'
-rw-r--r--  1 dada dada   4323432 Aug  3 13:52  All_companies3.xlsx
-rw-rw-r--  1 dada dada     26880 Aug  6 15:31  comp_ma.pkl
-rw-rw-r--  1 dada dada       216 Aug  7 18:19  comp_missing.csv
-rw-rw-r--  1 dada dada       783 Aug  9 12:32  comp_missing_.csv
-rw-rw-r--  1 dada dada       291 Aug  9 13:44  comp_missing__.csv
-rw-rw-r--  1 dada dada       192 Aug  9 14:09  comp_missing___.csv
-rw-rw-r--  1 dada dada      1738 Aug  9 12:05  comp_missing.pkl
-rw-r--r--  1 dada dada   1590537 Aug  4 23:09  Data_for_David.xlsx
-rw-r--r--  1 dada dada   1162719 Aug  3 14:27  Data_for_David.xlsx.bak
-rw-r--r--  1 dada dada    170765 Aug 10 10:29  Extract_SEC_filings.ipynb
drwxr-xr-x  2 dada dada      4096 Aug 10 11:57  .ipynb_checkpoints/
-rw-r--r--  1 dada dada   5675706 Aug  5 08:59 'LIST OF PUBLICLY T